In [1]:
import os
import json
import numpy as np
import pickle 
import tqdm
from scipy import sparse
from collections import defaultdict
from pprint import pprint
from collections import Counter
from itertools import permutations


# Decoder

Read steam api match and create match datapoint
 
```
for each team; predict_log_proba for each hero in team

append log_proba to

        pos

h1  p   p   p   p   p
h2  p   p   p   p   p
h3  p   p   p   p   p
h4  p   p   p   p   p
h5  p   p   p   p   p
```

Create permutations for team (0 to 4, r=5). Use itertools permutations
```
c = [1,3,0,4,2] // get the log_prob by index in pos matrix
log_p = sum(c)
if log_p > best_p:
    best_p = log_p
    best_c = c

``` 



In [4]:
with open("../data/steam_api_match_example.json", "r") as f:
    match = json.load(f)
    match = match["result"]

with open("../data/clfs/logreg_clfs_all.pkl", "rb") as f:
    clfs = pickle.load(f)

with open("../data/heroes.json", "r") as f:
    heroes = json.load(f)
    heroes_id_to_name = {hero["id"]: hero["localized_name"] for hero in heroes}

def is_valid_match(match):
    if "match_id" not in match:
        return False
    if not match["lobby_type"] in {0, 5, 6, 7}:
        print("Invalid lobby type:", match["lobby_type"])
        return False
    if match["duration"] < 60 * 20:
        print("Invalid duration:", match["duration"])
        return False
    if not match["game_mode"] in {1, 2, 16, 22}:
        print("Invalid game mode:", match["game_mode"])
        return False

    return True

def get_rank(rank: int):
    oh = np.zeros(5)
    oh[rank] = 1
    return oh

opendota_data_path = "../data/dataset_positions_all.pkl" 
with open(opendota_data_path, 'rb') as fp:
    data = pickle.load(fp)

def remove_unplayed_roles(hid, y_pred, threshold=200):
    ys = data["ys"]
    role_counts = dict(Counter(ys[hid]))
    updated_y_pred = np.zeros(y_pred.shape)

    for k in range(0,5):
        if role_counts[k + 1] < threshold:
            updated_y_pred[k] = -1000.
        else:
            updated_y_pred[k] = y_pred[k]
    return updated_y_pred


def find_optimal_roles(match):
    players = match["players"]
    t0 = [p for p in players if p["player_slot"] < 128]
    t1 = [p for p in players if p["player_slot"] >= 128]

    for team in [t0, t1]:
        # !Ranks
        attributes = ["gold_per_min", "xp_per_min", "kills", "deaths", "assists", "last_hits", "hero_damage", "tower_damage"]
        ranks = {attr: sorted([(p["hero_id"], p[attr]) for p in team], key=lambda x: x[1], reverse=True) for attr in attributes}
        hids = [p["hero_id"] for p in team]
        team_position_proba = defaultdict(list)
        
        # !Create c
        for p in team:
            features = []
            hid = p["hero_id"]

            teammates = np.zeros((136))
            for team_hid in hids:
                if team_hid != hid:
                    teammates[team_hid] = 1.
            
            features.append(teammates)
            
            for rank in ranks: 
                features.append(get_rank(ranks[rank].index((hid, p[rank]))))

            x = np.concatenate(features)
            y_pred = clfs[hid].predict_log_proba(x.reshape(1, -1))
            updated_y_pred = remove_unplayed_roles(hid, y_pred.ravel())         
            team_position_proba[hid] = updated_y_pred

        best_log_p = -np.inf
        best_comp = None

        # Optimal 
        for comp in permutations(range(5), 5):
            comp_with_heroid = [(comp[i], heroes_id_to_name[hid], round(team_position_proba[hid][comp[i]], 2)) for i, hid in enumerate(hids)]
            print(comp_with_heroid)
            log_p = np.array([team_position_proba[hid][comp[i]] for i, hid in enumerate(hids)]).sum()
            if log_p > best_log_p:
                best_log_p = log_p
                best_comp = sorted(comp_with_heroid, key=lambda x: x[0])
    


        # Greedy            

        # break
            # arg_max = np.argmax(team_position_proba[hid])
            
        # argmax team_position_proba
        # sett alt annet en pos til -inf



        # print("{} => {:.2f}".format(best_comp, best_log_p))

    return best_comp


if is_valid_match(match):
    comp = find_optimal_roles(match)
    print(comp)


/home/bao/anaconda3/envs/dota/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


[(0, 'Windranger', -1000.0), (1, 'Grimstroke', -1000.0), (2, 'Pudge', -0.39), (3, 'Clinkz', -1000.0), (4, 'Zeus', -2.44)]
[(0, 'Windranger', -1000.0), (1, 'Grimstroke', -1000.0), (2, 'Pudge', -0.39), (4, 'Clinkz', -1000.0), (3, 'Zeus', -0.88)]
[(0, 'Windranger', -1000.0), (1, 'Grimstroke', -1000.0), (3, 'Pudge', -2.38), (2, 'Clinkz', -1000.0), (4, 'Zeus', -2.44)]
[(0, 'Windranger', -1000.0), (1, 'Grimstroke', -1000.0), (3, 'Pudge', -2.38), (4, 'Clinkz', -1000.0), (2, 'Zeus', -3.51)]
[(0, 'Windranger', -1000.0), (1, 'Grimstroke', -1000.0), (4, 'Pudge', -3.76), (2, 'Clinkz', -1000.0), (3, 'Zeus', -0.88)]
[(0, 'Windranger', -1000.0), (1, 'Grimstroke', -1000.0), (4, 'Pudge', -3.76), (3, 'Clinkz', -1000.0), (2, 'Zeus', -3.51)]
[(0, 'Windranger', -1000.0), (2, 'Grimstroke', -1000.0), (1, 'Pudge', -1.83), (3, 'Clinkz', -1000.0), (4, 'Zeus', -2.44)]
[(0, 'Windranger', -1000.0), (2, 'Grimstroke', -1000.0), (1, 'Pudge', -1.83), (4, 'Clinkz', -1000.0), (3, 'Zeus', -0.88)]
[(0, 'Windranger', -1000